In [1]:
import pandas as pd
import urllib.request
from xml.etree.ElementTree import parse
import numpy as np

In [2]:
# 경기도 버스 노선 조회
# IlQuvwbMhQpoq5AqwRKt7iua3SCIVLL8FAs0kVHdoK8J7zHmn3QMof%2F2l6tsfD2fDy8ee2svGf%2FpCOkY8YCW%2FQ%3D%3D
# IlQuvwbMhQpoq5AqwRKt7iua3SCIVLL8FAs0kVHdoK8J7zHmn3QMof/2l6tsfD2fDy8ee2svGf/pCOkY8YCW/Q==

# [경기도 정류소 조회]
# IlQuvwbMhQpoq5AqwRKt7iua3SCIVLL8FAs0kVHdoK8J7zHmn3QMof%2F2l6tsfD2fDy8ee2svGf%2FpCOkY8YCW%2FQ%3D%3D
# IlQuvwbMhQpoq5AqwRKt7iua3SCIVLL8FAs0kVHdoK8J7zHmn3QMof/2l6tsfD2fDy8ee2svGf/pCOkY8YCW/Q==


In [3]:
bus = pd.read_csv('./data/버스정류소현황.csv')

is_bucheon = bus['시군명'] == '부천시'
bucheon_bus = bus[is_bucheon]
bucheon_bus = bucheon_bus.loc[:, ['시군명', '정류소명', '정류소id', 'WGS84위도', 'WGS84경도']]
display(bucheon_bus.head(5))
print(len(bucheon_bus))

,시군명,정류소명,정류소id,WGS84위도,WGS84경도
8611,부천시,KCC스위첸아파트.한신아파트,211000062,37.475267,126.799667
8612,부천시,KEB하나은행도당동지점,212000004,37.516817,126.779700
8613,부천시,KEB하나은행도당동지점,210000373,37.516600,126.779817
8614,부천시,KT부천지사,210000101,37.486967,126.776200
8615,부천시,KT부천지사,210000365,37.486867,126.775250


1162


In [12]:
columns = ['정류소id', '정류소명', '노선id', '최소배차', '최대배차', '첫차시간', '막차시간']
bus = pd.DataFrame({'정류소id' : [], '정류소명' : [], '노선id' : [], '최소배차' : [], '최대배차' : [], '첫차시간' : [], '막차시간' : []})

for i in range(len(bucheon_bus)):
    station_id = bucheon_bus.iloc[i, 2]
    station_name = bucheon_bus.iloc[i, 1]
    
    url_route = 'http://openapi.gbis.go.kr/ws/rest/busstationservice/route?serviceKey=1234567890&stationId=' + str(station_id)
    xml_route = urllib.request.urlopen(url_route)
    tree_route = parse(xml_route)
    root_route = tree_route.getroot()
    
    if root_route.find('msgHeader').findtext('resultMessage') != '정상적으로 처리되었습니다.':
        bus_tuple = (station_id, station_name, np.nan, np.nan, np.nan, np.nan, np.nan)
        bus_df = pd.DataFrame([bus_tuple], columns = columns)
        bus = bus.append(bus_df)
        continue
    
    for j in range(len(root_route.find('msgBody').findall('busRouteList'))):
        route_id = root_route.find('msgBody').findall('busRouteList')[j].findtext('routeId')
        if (route_id in list(bus['노선id'])) == True:
            min_bus = bus[bus['노선id'] == route_id]['최소배차'].iloc[0]
            max_bus = bus[bus['노선id'] == route_id]['최대배차'].iloc[0]
            first_bus = bus[bus['노선id'] == route_id]['첫차시간'].iloc[0]
            last_bus = bus[bus['노선id'] == route_id]['막차시간'].iloc[0]
        else:
            url_bus = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info?serviceKey=1234567890&routeId=' + route_id
            xml_bus = urllib.request.urlopen(url_bus)
            tree_bus = parse(xml_bus)
            root_bus = tree_bus.getroot()

            if root_bus.find('msgHeader').findtext('resultMessage') != '정상적으로 처리되었습니다.':
                continue

            min_bus = root_bus.find('msgBody').findall('busRouteInfoItem')[0].findtext('peekAlloc')
            max_bus = root_bus.find('msgBody').findall('busRouteInfoItem')[0].findtext('nPeekAlloc')
            first_bus = root_bus.find('msgBody').findall('busRouteInfoItem')[0].findtext('downFirstTime')
            last_bus = root_bus.find('msgBody').findall('busRouteInfoItem')[0].findtext('downLastTime')
            
        bus_tuple = (station_id, station_name, route_id, min_bus, max_bus, first_bus, last_bus)
        bus_df = pd.DataFrame([bus_tuple], columns = columns)
        bus = bus.append(bus_df)

In [13]:
bus

,정류소id,정류소명,노선id,최소배차,최대배차,첫차시간,막차시간
0,211000062.0,KCC스위첸아파트.한신아파트,210000073,13,17,05:20,23:15
0,211000062.0,KCC스위첸아파트.한신아파트,210000006,7,15,05:00,00:10
0,211000062.0,KCC스위첸아파트.한신아파트,210000004,5,10,05:15,00:00
0,211000062.0,KCC스위첸아파트.한신아파트,210000007,7,15,04:40,00:10
0,211000062.0,KCC스위첸아파트.한신아파트,210000038,7,10,05:45,00:00
...,...,...,...,...,...,...,...
0,212000395.0,휴먼시아1단지후문,210000099,20,30,06:25,23:25
0,212000378.0,휴먼시아3단지,212000005,8,10,05:25,23:35
0,212000378.0,휴먼시아3단지,210000099,20,30,06:25,23:25
0,212000384.0,휴먼시아3단지,212000005,8,10,05:25,23:35


In [14]:
bus.to_csv('output/bus_num.csv')

In [11]:
url_bus = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info?serviceKey=1234567890&routeId=' + str(2100000122)
xml_bus = urllib.request.urlopen(url_bus)
tree_bus = parse(xml_bus)
root_bus = tree_bus.getroot()
root_bus.find('msgHeader').findtext('resultMessage')

'결과가 존재하지 않습니다.'

In [ ]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

In [ ]:
url = 'http://apis.data.go.kr/6410000/busstationservice/getBusStationList'

queryParams = '?' + urlencode({
    quote_plus('ServiceKey') : 'taY2YBA5CmG2TFflQE%2FY2hhxY2PLpTQFzTbgKADwpcoRpQKsmQL6dOVjk6yK6MkwltHlVjQunqAWure67WsXUA%3D%3D',
    quote_plus('stationId') : '233001450'
})

print(url + queryParams)

#request = Request(url + queryParams)
#request.get_method = lambda: 'GET'
#response_body = urlopen(request).read()
#print(response_body)

In [ ]:
url = 'http://apis.data.go.kr/6410000/busrouteservice/getBusRouteInfoItem'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : 'taY2YBA5CmG2TFflQE%2FY2hhxY2PLpTQFzTbgKADwpcoRpQKsmQL6dOVjk6yK6MkwltHlVjQunqAWure67WsXUA%3D%3D',
                               quote_plus('routeId') : '200000085' })

print(url + queryParams)
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read()
print(response_body)